# COPY INTO and MERGE Commands

### COPY INTO Command

> - Incrementally loads data into Delta Lake tables from Cloud Storage  
> - Supports schema evolution  
> - Supports wide range of file formats (CSV, JSON, Parquet, Delta)  
> - Alternative to Auto Loader for batch ingestion  

Documentation - [COPY INTO](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-copy-into)

In [0]:
CREATE TABLE IF NOT EXISTS demo.delta_lake.raw_stock_prices;

In [0]:
DELETE FROM demo.delta_lake.raw_stock_prices;

COPY INTO demo.delta_lake.raw_stock_prices
FROM 'abfss://demo@deacourseexternaldl100.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema'= 'true')
COPY_OPTIONS ('mergeSchema'= 'true')

In [0]:
select * from demo.delta_lake.raw_stock_prices

In [0]:
create table if not exists demo.delta_lake.stock_prices
(
  stock_id STRING,
  price DOUBLE,
  trading_date DATE 
)

In [0]:
MERGE INTO demo.delta_lake.stock_prices AS target
USING demo.delta_lake.raw_stock_prices AS source
ON target.stock_id = source.stock_id
WHEN MATCHED AND source.status = 'ACTIVE' THEN
  UPDATE SET target.price = source.price, target.trading_date = source.trading_date
WHEN MATCHED AND source.status = 'DELISTED' THEN
  DELETE
WHEN NOT MATCHED AND source.status = 'ACTIVE' THEN
  INSERT (stock_id, price, trading_date) VALUES(source.stock_id, source.price, source.trading_date)

In [0]:
SELECT * FROM demo.delta_lake.stock_prices